<a href="https://colab.research.google.com/github/cynthia087/Supply_Chain_Analysis/blob/main/BiopharmaCase/biopharmatesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal

# Define the API URL and currency pairs
API_URL = "https://api.frankfurter.app"
BASE_CURRENCY = "USD"
TARGET_CURRENCIES = ["BRL", "EUR", "INR", "JPY", "MXN"]
YEARS = list(range(2013, 2005, -1))  # Get rates for 2013 to 2006

# Function to fetch exchange rates for a given date
def get_exchange_rate_for_date(date_str, base_currency, symbols):
    url = f"{API_URL}/{date_str}?from={base_currency}&to={','.join(symbols)}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()["rates"]
    else:
        print(f"Error fetching data for {date_str}: {response.status_code}")
        return None

# Fetch historical exchange rates
exchange_rates = []
for year in YEARS:
    date_str = f"{year}-01-31"  # Use January 31st as a reference date
    print(f"Fetching rates for {date_str}...")
    rates = get_exchange_rate_for_date(date_str, BASE_CURRENCY, TARGET_CURRENCIES)
    if rates:
        rates["Year"] = year
        exchange_rates.append(rates)

# Convert to DataFrame
exchange_rate_df = pd.DataFrame(exchange_rates)

# Drop any rows with missing values
exchange_rate_df.dropna(inplace=True)

# Display first few rows of fetched exchange rates
print("\nFetched Exchange Rates (After Dropping NaNs):\n", exchange_rate_df.head())

# Extract numeric exchange rate values
exchange_rate_values = exchange_rate_df.iloc[:, :-1].values  # Exclude "Year"

# Compute mean and covariance matrix
mean_rates = np.nanmean(exchange_rate_values, axis=0)  # Use nanmean to handle NaNs
cov_rates = np.cov(exchange_rate_values.T)

# Replace NaNs or Infs in covariance matrix
cov_rates = np.nan_to_num(cov_rates, nan=1e-6, posinf=1e-6, neginf=1e-6)

# Fix covariance matrix if not positive semi-definite
eigvals = np.linalg.eigvals(cov_rates)
if np.any(eigvals < 0):
    cov_rates += np.eye(cov_rates.shape[0]) * 1e-6  # Add small value to diagonal

# Generate 100 exchange rate scenarios using multivariate normal distribution
np.random.seed(42)  # Ensure reproducibility
samples = np.random.multivariate_normal(mean_rates, cov_rates, 100)

# Convert samples to DataFrame
exchange_rate_samples = pd.DataFrame(samples, columns=exchange_rate_df.columns[:-1])

# Display first few rows of generated samples
print("\nGenerated 100 Exchange Rate Samples:\n", exchange_rate_samples.head())


Fetching rates for 2013-01-31...
Fetching rates for 2012-01-31...
Fetching rates for 2011-01-31...
Fetching rates for 2010-01-31...
Fetching rates for 2009-01-31...
Fetching rates for 2008-01-31...
Fetching rates for 2007-01-31...
Fetching rates for 2006-01-31...

Fetched Exchange Rates (After Dropping NaNs):
       BRL      EUR     INR     JPY      MXN  Year
0  1.9846  0.73801  53.225  91.010  12.7065  2013
1  1.7375  0.75896  49.435  76.374  12.9200  2012
2  1.6770  0.73035  45.908  82.160  12.1543  2011
3  1.8621  0.71602  46.195  90.330  13.0250  2010
4  2.3127  0.78027  48.690  89.720  14.3750  2009

Generated 100 Exchange Rate Samples:
         BRL       EUR        INR        JPY        MXN
0  2.027458  0.774853  48.707378  82.658860  13.357383
1  2.000102  0.741823  44.337440  87.921889  13.697997
2  1.958756  0.738096  50.372919  88.726201  13.385456
3  2.061213  0.752394  52.013149  89.172664  13.458715
4  1.618848  0.742009  48.236466  76.439085  12.652683


In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 57.1 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd
import numpy as np
from tabulate import tabulate  # For displaying results in a table

# Define plant names
plants = ["Brazil", "Germany", "India", "Japan", "Mexico", "US"]

# Fixed and variable costs (From Case Study)
fixed_costs = {
    "Brazil": 20.0, "Germany": 45.0, "India": 14.0,
    "Japan": 13.0, "Mexico": 30.0, "US": 23.0
}
variable_costs = {
    "Brazil": 5.1, "Germany": 6.0, "India": 4.5,
    "Japan": 6.0, "Mexico": 5.0, "US": 5.0
}

# Map plant locations to exchange rate currencies
currency_mapping = {
    "Brazil": "BRL",
    "Germany": "EUR",
    "India": "INR",
    "Japan": "JPY",
    "Mexico": "MXN",
    "US": "USD"  # USD does not need conversion
}

# Plant capacities
plant_capacities = {"Brazil": 18, "Germany": 45, "India": 18, "Japan": 10, "Mexico": 30, "US": 22}

# Adjust exchange rate variability
scaled_cov_rates = cov_rates * 6  # More extreme rate shifts
exchange_rate_samples = np.random.multivariate_normal(mean_rates, scaled_cov_rates, 100)
exchange_rate_samples = pd.DataFrame(exchange_rate_samples, columns=["BRL", "EUR", "INR", "JPY", "MXN"])

# Store optimization results
scenario_results = []

# Run optimization for each exchange rate sample
for i in range(len(exchange_rate_samples)):
    rates = exchange_rate_samples.iloc[i]

    # Convert costs to USD
    cost_usd = {}
    for plant in plants:
        currency = currency_mapping[plant]
        if currency == "USD":
            cost_usd[plant] = fixed_costs[plant] + variable_costs[plant]  # No conversion for USD
        else:
            cost_usd[plant] = (fixed_costs[plant] + variable_costs[plant]) / rates[currency]

    # Print cost breakdown for debugging
    print(f"\nScenario {i+1} Cost in USD:")
    for plant, cost in cost_usd.items():
        print(f"{plant}: ${cost:.2f}")

    # Increase demand range to force more plants to open
    demand = np.random.randint(100, 115)

    # Initialize Gurobi Model
    model = gp.Model("Biopharma_Optimization")

    # Decision variables: x[p] = 1 if plant p is open, 0 otherwise
    x = model.addVars(plants, vtype=GRB.BINARY, name="OpenPlant")

    # Objective: Minimize total cost with a **smaller penalty** for opening plants
    model.setObjective(
        quicksum(cost_usd[p] * x[p] for p in plants) + quicksum(250 * x[p] for p in plants),  # Lower penalty
        GRB.MINIMIZE
    )

    # Constraint: Ensure demand is met
    model.addConstr(quicksum(plant_capacities[p] * x[p] for p in plants) >= demand, "MeetDemand")

    # Solve model
    model.optimize()

    # Store results: Which plants remain open
    open_plants = [p for p in plants if x[p].x > 0.5]
    total_cost = model.objVal if model.status == GRB.OPTIMAL else None
    scenario_results.append({"Scenario": i+1, "Open_Plants": open_plants, "Total_Cost": total_cost})

# Convert results to DataFrame
plant_open_df = pd.DataFrame(scenario_results)



Scenario 1 Cost in USD:
Brazil: $31.24
Germany: $78.51
India: $0.55
Japan: $0.26
Mexico: $3.60
US: $28.00
Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 6 columns and 6 nonzeros
Model fingerprint: 0xa197ea45
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+01, 4e+01]
  Objective range  [3e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective 1110.3723223
Presolve removed 1 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 1110

In [ ]:
# Prepare results in a structured table format
table_data = []
for index, row in plant_open_df.iterrows():
    row_data = [row["Scenario"], ", ".join(row["Open_Plants"]), f"${row['Total_Cost']:,.2f}"]
    table_data.append(row_data)

# Display results using tabulate
headers = ["Scenario", "Open Plants", "Total Cost (USD)"]
print(tabulate(table_data[:10], headers=headers, tablefmt="grid"))  # Show first 10 scenarios


+------------+--------------------------------+--------------------+
|   Scenario | Open Plants                    | Total Cost (USD)   |
+============+================================+====================+
|          1 | Germany, Japan, Mexico, US     | $1,110.37          |
+------------+--------------------------------+--------------------+
|          2 | Brazil, Germany, India, Mexico | $1,088.96          |
+------------+--------------------------------+--------------------+
|          3 | Germany, India, Japan, Mexico  | $1,074.45          |
+------------+--------------------------------+--------------------+
|          4 | Brazil, Germany, India, Mexico | $1,099.73          |
+------------+--------------------------------+--------------------+
|          5 | Brazil, Germany, India, Mexico | $1,087.07          |
+------------+--------------------------------+--------------------+
|          6 | Germany, India, Mexico, US     | $1,096.53          |
+------------+--------------------

In [ ]:
# Count how often each plant remains open
plant_frequencies = pd.DataFrame(columns=plants)
for plant in plants:
    plant_frequencies[plant] = plant_open_df["Open_Plants"].apply(lambda x: 1 if plant in x else 0)

# Calculate opening probability for each plant
plant_opening_prob = plant_frequencies.mean()

# Identify plants that open/close together
correlation_matrix = plant_frequencies.corr()

# Display results
print("\nPlant Opening Probabilities:")
print(plant_opening_prob)

print("\nCorrelation Between Plant Openings:")
print(correlation_matrix)



Plant Opening Probabilities:
Brazil     0.52
Germany    1.00
India      0.97
Japan      0.22
Mexico     1.00
US         0.29
dtype: float64

Correlation Between Plant Openings:
           Brazil  Germany     India     Japan  Mexico        US
Brazil   1.000000      NaN  0.183044 -0.552771     NaN -0.665198
Germany       NaN      NaN       NaN       NaN     NaN       NaN
India    0.183044      NaN  1.000000 -0.331139     NaN -0.275172
Japan   -0.552771      NaN -0.331139  1.000000     NaN -0.179817
Mexico        NaN      NaN       NaN       NaN     NaN       NaN
US      -0.665198      NaN -0.275172 -0.179817     NaN  1.000000


In [ ]:
average_costs = {plant: np.mean([cost_usd[plant] for i in range(len(exchange_rate_samples))]) for plant in plants}
print("\nAverage Costs Across Scenarios:")
for plant, cost in sorted(average_costs.items(), key=lambda x: x[1]):
    print(f"{plant}: ${cost:.2f}")



Average Costs Across Scenarios:
Japan: $0.21
India: $0.36
Mexico: $2.73
Brazil: $12.90
US: $28.00
Germany: $69.61


STRATEGY 1: Keep Germany, India, Mexico open, as they are always used. Japan is open in some cases (~67%), so we include Japan. Brazil and the US remain closed as they are less frequently selected.

STRATEGY 2: Same as Strategy 1, but include Brazil, which opens ~53% of the time. US remains closed, as it is still rarely chosen.

STRATEGY 3: Since Germany had a very high cost, we try an alternative strategy without Germany. We replace Germany with Brazil and the US, spreading production across cheaper plants.

In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum
from tabulate import tabulate

# Define the three fixed strategies
strategies = {
    "Strategy 1 (Core)": ["Germany", "India", "Mexico", "Japan"],
    "Strategy 2 (Expanded)": ["Germany", "India", "Mexico", "Japan", "Brazil"],
    "Strategy 3 (Alternative, No Germany)": ["Brazil", "India", "Mexico", "Japan", "US"]
}

# Generate 100 new exchange rate samples
scaled_cov_rates = cov_rates * 6  # Maintain high variability
exchange_rate_samples = np.random.multivariate_normal(mean_rates, scaled_cov_rates, 100)
exchange_rate_samples = pd.DataFrame(exchange_rate_samples, columns=["BRL", "EUR", "INR", "JPY", "MXN"])

# Storage for strategy performance
# Adjust strategies to ensure enough capacity
strategies["Strategy 1 (Core)"].append("Brazil")  # Adds +18 capacity, making it feasible
strategies["Strategy 3 (Alternative, No Germany)"].append("Germany")  # Adds +35 capacity, making it feasible

# Re-run evaluation with fixed strategies
strategy_results = []

for strategy_name, open_plants in strategies.items():
    total_costs = []

    for i in range(len(exchange_rate_samples)):
        rates = exchange_rate_samples.iloc[i]

        # Convert costs to USD (but plants are now fixed!)
        cost_usd = {}
        for plant in open_plants:
            currency = currency_mapping[plant]
            if currency == "USD":
                cost_usd[plant] = fixed_costs[plant] + variable_costs[plant]
            else:
                cost_usd[plant] = (fixed_costs[plant] + variable_costs[plant]) / rates[currency]

        # Set demand range (lower if necessary)
        demand = np.random.randint(110, 125)

        # Initialize Gurobi Model (Fixed Plants)
        model = gp.Model(f"Biopharma_Evaluation_{strategy_name}")

        # Decision variables: Production allocation (but plants are fixed)
        x = model.addVars(open_plants, vtype=GRB.CONTINUOUS, name="Production")

        # Objective: Minimize total cost (only optimizing production)
        model.setObjective(quicksum(cost_usd[p] * x[p] for p in open_plants), GRB.MINIMIZE)

        # Constraint: Ensure demand is met (Relaxed)
        model.addConstr(quicksum(x[p] for p in open_plants) >= demand - 5, "RelaxedDemand")

        # Capacity Constraints
        for plant in open_plants:
            model.addConstr(x[plant] <= plant_capacities[plant], f"Capacity_{plant}")

        # Solve model
        model.optimize()

        # Store total cost
        if model.status == GRB.OPTIMAL:
            total_costs.append(model.objVal)

    # Compute expected cost and standard deviation
    expected_cost = np.mean(total_costs)
    cost_std_dev = np.std(total_costs)

    strategy_results.append({
        "Strategy": strategy_name,
        "Expected Cost": expected_cost,
        "Cost Std Dev": cost_std_dev
    })

# Convert to DataFrame
strategy_results_df = pd.DataFrame(strategy_results)

# Display results
print(tabulate(strategy_results_df, headers="keys", tablefmt="grid"))


Streaming output truncated to the last 5000 lines.
  Objective range  [3e-01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 6 rows and 5 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.2553675e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.255367513e+03
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x9e88f491
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 6 rows and 5 columns
Presolve

In [ ]:
for strategy_name, open_plants in strategies.items():
    total_capacity = sum(plant_capacities[p] for p in open_plants)
    print(f"{strategy_name}: Total Capacity = {total_capacity}")


Strategy 1 (Core): Total Capacity = 121
Strategy 2 (Expanded): Total Capacity = 121
Strategy 3 (Alternative, No Germany): Total Capacity = 143
